In [0]:
import numpy as np
import math
import os.path
import csv
import glob
import tensorflow as tf
import h5py as h5py

from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model

Using TensorFlow backend.


In [0]:
import tensorflow
tensorflow.__version__

'2.2.0-rc3'

In [0]:
# Package for face landmark detection
# !pip install mlxtend
import mlxtend

In [0]:
import mlxtend
import matplotlib.pyplot as plt
from mlxtend.image import extract_face_landmarks

# New Section

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
! ls '/content/drive/My Drive/CSCE_689'

 0.avi	   6.avi				      Labels70-73.csv
 100.avi   70.avi				      Labels7-10.csv
 10.avi    71.avi				      Labels_final.csv
 11.avi    72.avi				      labels.npy
 12.avi    73.avi				      LR.sav
 13.avi    74.avi				      MLP.sav
 14.avi    75.avi				      model1.h5
 15.avi    76.avi				      model.h5
 16.avi    77.avi				      model.json
 17.avi    78.avi				      model.png
 18.avi    79.avi				      NN_test.npy
 19.avi    7.avi				      NN_train.npy
 1.avi	   80.avi				      optimization
 20.avi    81.avi				      Prediction
 21.avi    82.avi				      Prediction_train
 22.avi    83.avi				     'Pred_label (1).npy'
 23.avi    84.avi				      Pred_label.npy
 24.avi    85.avi				      README.md
 25.avi    86.avi				      RF.sav
 26.avi    87.avi				      Submision_1
 27.avi    88.avi				      SVM.sav
 28.avi    89.avi				      Test
 29.avi    8.avi				      Test_1.avi
 2.avi	   90.avi				     'Test1 - NN.png'
 30.avi    91.avi				     'Test1 - TL.png'
 31.avi    92.avi				      Test

In [0]:
cd '/content/drive/My Drive/CSCE_689/'

/content/drive/.shortcut-targets-by-id/188Eba_cLfN68Gy_4d0czpAJtiCmTs8jW/CSCE_689


### Define and compute inputs terms 

In [0]:
def eye_aspect_ratio(eye):
	
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
	return ear

In [0]:
def mouth_aspect_ratio(mouth):
    A = distance.euclidean(mouth[14], mouth[18])
    C = distance.euclidean(mouth[12], mouth[16])
    mar = (A ) / (C)
    return mar

In [0]:
def circularity(eye):
    A = distance.euclidean(eye[1], eye[4])
    radius  = A/2.0
    Area = math.pi * (radius ** 2)
    p = 0
    p += distance.euclidean(eye[0], eye[1])
    p += distance.euclidean(eye[1], eye[2])
    p += distance.euclidean(eye[2], eye[3])
    p += distance.euclidean(eye[3], eye[4])
    p += distance.euclidean(eye[4], eye[5])
    p += distance.euclidean(eye[5], eye[0])
    return 4 * math.pi * Area /(p**2)

In [0]:
def mouth_over_eye(eye):
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(eye)
    mouth_eye = mar/ear
    return mouth_eye

In [0]:
def jaw_dist(jaw,nose):
  list= np.array([distance.euclidean(d, nose) for d in jaw])
  return list.max()/list.mean()



In [0]:
def getFrame(sec):
    start = 0
    vidcap.set(cv2.CAP_PROP_POS_MSEC, start + sec*1000)
    hasFrames,image = vidcap.read()
    return hasFrames, image

In [0]:
def get_landmarks(vidcap,i):
  sec = 0
  frameRate = 0.05
  success, image = getFrame(sec)
  count = 0
  data=[]
  labels=[]
  timestamp=[]
  while success and count < int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)):
      landmarks = extract_face_landmarks(image)
      try:
          sum(sum(landmarks))
          if sum(sum(landmarks)) != 0:
              count += 1
              data.append(landmarks)
              labels.append([i])
              # cv2.imwrite("frame%d.jpg" % count, image)
              timestamp.append(sec)
              sec = sec + frameRate
              sec = round(sec, 3)
              success, image = getFrame(sec)

              # print('success: '+ str(success)+str(int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))))
          else:
              sec = sec + frameRate
              sec = round(sec, 3)
              success, image = getFrame(sec)
              print(str(i)+' not detected')
      except:
          # print('success: '+ str(success)+str(int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))))
          count=int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))
          pass
      # print(count)
  return data,labels,timestamp

In [0]:
# Generate Landmarks for all training/validation videos
from scipy.spatial import distance
import cv2
data = []
labels = []
frmes=[]
timestamp=[]
for i in range(101):
    vidcap = cv2.VideoCapture(str(i) + '.avi')
    data.extend(get_landmarks(vidcap,i)[0])
    labels.extend(get_landmarks(vidcap,i)[1])
    timestamp.extend(get_landmarks(vidcap,i)[2])
   
        

/usr/local/lib/python3.6/dist-packages/mlxtend/image/extract_face_landmarks.py:61: UserWarning: No face detected.
  warnings.warn('No face detected.')


70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not detected
70 not d

In [0]:
##Create Features
import numpy as np
import math
data = np.array(data)
features = []
main_list=[]
jaw_list=[]
for d in data:
    eye = d[36:68]
    nose=d[28]
    jaw=d[5:13]
    main_list.append(eye)
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(eye)
    cir = circularity(eye)
    movt = jaw_dist(jaw,nose)
    mouth_eye = mouth_over_eye(eye)
    features.append([ear, mar, cir, mouth_eye,movt])

In [0]:
import pandas as pd
features = np.array(features)
features.shape
time= np.array(timestamp)
x=pd.DataFrame()
x['label']=labels
x['timestamp']=time

In [0]:
# Save files
np.save(open('Data.npy', 'wb'),data) ## Save landmark data
np.savetxt('Timestamps.csv',time)
np.savetxt("Features_final.csv", features, delimiter = ",")
np.savetxt("Labels.csv", labels, delimiter = ",")

In [0]:
## Save a combine file of label and timestamp. It will be converted to Labels_final.csv which will have labels of 0 (for not yawning) and 1 (for yawning)
x.to_csv("combinedlabels.csv") 

In [0]:
# Loading the files 
import numpy as np
import pandas as pd
labels=pd.read_csv('Labels_final.csv',header=None)
features=pd.read_csv('Features_final.csv',header=None)

In [0]:
# time values in each videos where yawning happens
time=[
 [4,9,14,18],
 [3,6,10.5,15],
 [0,0],
 [2,6],
 [4.5,7,11.5,16],
 [24,31,71,77],
 [38,41,59,63],
 [13,16,40,46],
 [7,11,34,37],
 [6,11.5],
 [3,6,14,20],
 [0.5,5,8,14],
 [1,6],
 [3,9],
 [4.5,8,10.5,13],
 [0,6,10,15],
 [4,8,17.5,20],
 [1,7],
 [4.5,10],
 [3.5,8],
 [4.5,7.5],
 [7.5,13],
 [16,20.5,31.5,34.5,44,49,62,64],
 [24.5,32,53,60,78,87],
 [1,5],
 [23,29,51,57,75,81],
 [4,5,8,12],
 [9,11,19.5,24,58,62],
 [11,16],
 [35,40,49,55,67,73],
 [3,10],
 [14,19,39,45,66,71,89,97],
 [2,5],
 [14,17,26,29,41,46],
 [1,6],
 [3,12],
 [0,5,6,8],
 [3,6,14,19],
 [3,7,14,19],
 [0,2,4.5,6],
 [0,4,10.5,16],
 [4.5,7,10.5,14],
 [34.5,40,49,55,67,73],
 [11,15,27,33,49,54],
 [33,36,38,40,67,70],
 [14,19,37.5,42,52,57],
 [39,41,56.5,61],
 [20,25,33.5,37,52,57],
 [23.5,26,29.5,32,41.5,44,56.5,60],
 [31,35,36,38,58.5,62],
 [8.5,10,19,22,44.5,49],
 [12,15,32,35],
 [3.5,8,12,17,27,34],
 [20,23,43,46,58,61,112,115],
 [21,25,35,39,45,48,66,71],
 [24,25,37,39],
 [0,0],
 [10,13,24,28,42,46],
 [53,56],
 [5,8],
 [1,5],
 [0,5,10,16],
 [2,7,14,19],
 [2,4,9,13],
 [2,5,13,18],
 [1,3,7,11],
 [4.5,12,16.5,23],
 [5.5,11,19.5,27],
 [1,5],
 [7,15],
 [4,8],
 [3.5,8],
 [1,5,8,11],
 [5,8],
 [4,9],
 [4,10],
 [7.5,11],
 [9,12],
 [9,12],
 [2,5],
 [6,9,19,22],
 [4,8],
 [3,7],
 [2,10,15,27,28,30],
 [5.5,10],
 [7,9],
 [4,7],
 [1,4],
 [5,9,11,14,17,19],
 [7,10,16,19],
 [6,9],
 [4,9],
 [0,5,9,12],
 [5,8],
 [3,6],
 [5,7],
 [15.5,18],
 [7,10,17.5,20],
 [2,5],
 [10,11.5],
 [6,8] 
]

In [0]:
## Generating train images for transfer learning
import cv2
j=0
c_a=0
c_b=0
for i in range(81):
  vidcap = cv2.VideoCapture(str(i) + '.avi')
  print(i)
  sec = 0
  frameRate = 0.05
  success, image = getFrame(sec)
  count = 0
  labels=[]
  while success and count <= int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)):
    it=iter(time[i])
    for x in it:
      if sec>= x and sec<=next(it):
        label=1
        # print(label)
        break
      else:
        label=0
        #  print(label)
    # print(str(label)+ str(sec))
    if label==0:
      # print('0')
      c_a=c_a+1
      cv2.imwrite('/content/drive/My Drive/CSCE_689/Train/Class_0/'+format(c_a)+str(".jpg"), image)
    else:
      # print('1')
      c_b=c_b+1
      cv2.imwrite('/content/drive/My Drive/CSCE_689/Train/Class_1/'+format(c_b)+str(".jpg"), image)
    sec = sec + frameRate
    sec = round(sec, 3)
    success, image = getFrame(sec)
    j=j+1
    count+=1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


In [0]:
## Generating test images for transfer learning
import cv2
j=0
c_a=0
c_b=0
labels=[]
timesta=[]
vid=[]
for i in range(81,101):
  vidcap = cv2.VideoCapture(str(i) + '.avi')
  print(i)
  sec = 0
  frameRate = 0.05
  success, image = getFrame(sec)
  count = 0
  while success and count <= int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)):
    it=iter(time[i])
    for x in it:
      if sec>= x and sec<=next(it):
        label=1
        # print(label)
        break
      else:
        label=0
        #  print(label)
    # print(str(label)+ str(sec))
    labels.append(label)
    timesta.append(sec)
    vid.append(i)
    if label==0:
      # print('0')
      c_a=c_a+1
      cv2.imwrite('/content/drive/My Drive/CSCE_689/Test/Class_0/'+format(c_a)+str(".jpg"), image)
    else:
      # print('1')
      c_b=c_b+1
      cv2.imwrite('/content/drive/My Drive/CSCE_689/Test/Class_1/'+format(c_b)+str(".jpg"), image)
    sec = sec + frameRate
    sec = round(sec, 3)
    success, image = getFrame(sec)
    j=j+1
    count+=1


81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [0]:
np.save(open('time.npy', 'wb'),np.array(timesta)) ## Contains full time for 0-100 videos
np.save(open('vid.npy', 'wb'),np.array(vid)) ## Contains full vid for 0-100 videos


In [0]:
## Generating prediction train images for transfer learning
import cv2
j=0
c_a=100000
c_b=0
labels=[]
train_vid=[]
train_time=[]
for i in range(81):
  vidcap = cv2.VideoCapture(str(i) + '.avi')
  print(i)
  sec = 0
  frameRate = 0.05
  success, image = getFrame(sec)
  count = 0
  while success and count <= int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)):
    c_a=c_a+1

    # cv2.imwrite('/content/drive/My Drive/CSCE_689/Prediction_train/'+format(c_a)+str(".jpg"), image)
    train_vid.append(i)
    train_time.append(sec)
    sec = sec + frameRate
    sec = round(sec, 3)
    success, image = getFrame(sec)
    j=j+1
    count+=1


In [0]:
# Desired to change to the code below, but not implemented
train_tl_df=pd.DataFrame()
train_tl_df["vid"]=train_vid
train_tl_df["time"]=train_time
train_tl_df.to_csv("TL_train.csv")

In [0]:
## Generating prediction test images for transfer learning
import cv2
j=0
c_a=100000
c_b=0
labels=[]
test_vid=[]
test_time=[]
for i in range(81,101):
    vidcap = cv2.VideoCapture(str(i) + '.avi')
    print(i)
    sec = 0
    frameRate = 0.05
    success, image = getFrame(sec)
    count = 0
    while success and count <= int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)):
        c_a=c_a+1

        cv2.imwrite('/content/drive/My Drive/CSCE_689/Prediction/'+format(c_a)+str(".jpg"), image)
        test_time.append(sec)
        test_vid.append(i)
        sec = sec + frameRate
        sec = round(sec, 3)
        success, image = getFrame(sec)
        j=j+1
        count+=1


81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [0]:
# Desired to change to the code below, but not implemented
test_tl_df=pd.DataFrame()
test_tl_df["vid"]=test_vid
test_tl_df["time"]=test_time
test_tl_df.to_csv("TL_test.csv")

In [0]:
len(np.load("time.npy"))

69392

In [0]:
## Generating all prediction test images for transfer learning
import cv2
j=0
c_a=100000
c_b=0
labels=[]
all_vid=[]
all_time=[]
for i in range(101):
    vidcap = cv2.VideoCapture(str(i) + '.avi')
    print(i)
    sec = 0
    frameRate = 0.05
    success, image = getFrame(sec)
    count = 0
    while success and count <= int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)):
        c_a=c_a+1

        cv2.imwrite('/content/drive/My Drive/CSCE_689/All/'+format(c_a)+str(".jpg"), image)
        all_time.append(sec)
        all_vid.append(i)
        sec = sec + frameRate
        sec = round(sec, 3)
        success, image = getFrame(sec)
        j=j+1
        count+=1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
